# Suivi de Trajectoire

In [ ]:
#pip install pygame

In [1]:
import math
import matplotlib.pyplot as plt
import pygame
import random
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
'''
class cible:
    trajecory = []
    def __init__(self, speed_0, position_0, acceleration_0):
        self.speed = speed_0
        self.position = position_0
        self.acceleration = acceleration_0
    def trajectoire(self):
        print(self.trajectory)
'''

'\nclass cible:\n    trajecory = []\n    def __init__(self, speed_0, position_0, acceleration_0):\n        self.speed = speed_0\n        self.position = position_0\n        self.acceleration = acceleration_0\n    def trajectoire(self):\n        print(self.trajectory)\n'

In [2]:
del_t = 1
k_x = 1
k_v = 1

In [17]:
class drone:
    trajectory = []
    def __init__(self, speed_0, position_0, acceleration_0, altitude_0):
        self.speed = speed_0
        self.position = position_0
        self.acceleration = acceleration_0
        self.altitude = altitude_0
        self.color = (120, 30, 190)
        self.rect = pygame.Rect(self.position[0], self.position[1], 32, 32)
        self.max_speed = np.array([0.5, 0.5])

    def radius(self):
        return 10
    
    def draw(self, win):
        pygame.draw.rect(win, self.color, self.rect)

    def update(self, target_position, target_speed):
        del_x = target_position - self.position
        del_v = target_speed - self.speed
        self.acceleration = k_x*del_x + k_v*del_v
        
        self.speed = self.speed + self.acceleration*del_t

        self.speed = np.maximum(-self.max_speed, np.minimum(self.max_speed, self.speed))
        self.speed = self.speed + np.random.normal(loc=0.0, scale=0.1, size=2)
        self.position = self.position + self.speed*del_t

        self.rect = pygame.Rect(int(self.position[0]), int(self.position[1]), 32, 32)

In [18]:
#Player Class
class Player:
    def __init__(self, x, y):
        self.position = np.array([x, y])
        self.rect = pygame.Rect(self.position[0], self.position[1], 32, 32)
        self.color = (250, 120, 60)
        self.speed = np.array([0, 0])
        self.left_pressed = False
        self.right_pressed = False
        self.up_pressed = False
        self.down_pressed = False
        self.pace = 1
    
    def draw(self, win):
        pygame.draw.rect(win, self.color, self.rect)
    
    def update(self):
        self.speed = np.array([0, 0])
        if self.left_pressed and not self.right_pressed:
            self.speed[0] = -self.pace
        if self.right_pressed and not self.left_pressed:
            self.speed[0] = self.pace
        if self.up_pressed and not self.down_pressed:
            self.speed[1] = -self.pace
        if self.down_pressed and not self.up_pressed:
            self.speed[1] = self.pace

        self.position[0] += self.speed[0]*del_t
        self.position[1] += self.speed[1]*del_t

        self.rect = pygame.Rect(int(self.position[0]), int(self.position[1]), 32, 32)

In [19]:
"""Pygame window initialisation"""

#Imports
import sys

#Constants
WIDTH, HEIGHT = 800, 800
TITLE = "Drone pursuit"

#pygame initialization
pygame.init()
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption(TITLE)
clock = pygame.time.Clock()

In [20]:
#Object initialisation
player = Player(WIDTH/2, HEIGHT/2)
drone = drone(np.array([WIDTH/4, HEIGHT/4]), np.zeros(2), np.zeros(2), 0)

In [21]:
#Main Loop
while True:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                player.left_pressed = True
            if event.key == pygame.K_RIGHT:
                player.right_pressed = True
            if event.key == pygame.K_UP:
                player.up_pressed = True
            if event.key == pygame.K_DOWN:
                player.down_pressed = True
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                player.left_pressed = False
            if event.key == pygame.K_RIGHT:
                player.right_pressed = False
            if event.key == pygame.K_UP:
                player.up_pressed = False
            if event.key == pygame.K_DOWN:
                player.down_pressed = False
        
    #Draw
    win.fill((12, 24, 36))  
    player.draw(win)
    drone.draw(win)
    
    #update
    player.update()
    drone.update(player.position, player.speed)
    pygame.display.flip()

    clock.tick(120)

SystemExit: 